In [1]:
# huggingface login
# %pip install -r /share/pi/nigam/users/aunell/mistral-finetune/requirements.dev.txt
# %pip install ipywidgets
from huggingface_hub import notebook_login

notebook_login()

In [2]:
%pip install huggingface_hub
from huggingface_hub import snapshot_download
from pathlib import Path
import os

mistral_models_path = "/share/pi/nigam/users/aunell/mistral-finetune/mistral_models/7B-v0.3"
os.makedirs(mistral_models_path, exist_ok=True)
# mistral_models_path.mkdir(parents=True, exist_ok=True)
print(mistral_models_path)
snapshot_download(repo_id="mistralai/Mistral-7B-Instruct-v0.3", allow_patterns=["params.json", "consolidated.safetensors", "tokenizer.model.v3"], local_dir=mistral_models_path)

# ! cp -r /root/mistral_models/7B-v0.3 /content/mistral_models
# ! rm -r /root/mistral_models/7B-v0.3
%pwd

Note: you may need to restart the kernel to use updated packages.
/share/pi/nigam/users/aunell/mistral-finetune/mistral_models/7B-v0.3


Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

'/share/pi/nigam/users/aunell/mistral-finetune'

In [3]:
%ls /share/pi/nigam/users/aunell/mistral-finetune/mistral_models/7B-v0.3
%mkdir -p /share/pi/nigam/users/aunell/mistral-finetune/data

consolidated.safetensors  params.json  tokenizer.model.v3


In [5]:
# read data into a pandas dataframe
%pip install pyarrow
import pandas as pd

df = pd.read_parquet('https://huggingface.co/datasets/HuggingFaceH4/ultrachat_200k/resolve/main/data/test_gen-00000-of-00001-3d4cd8309148a71f.parquet')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 MB 70.1 MB/s eta 0:00:00:00:01
Note: you may need to restart the kernel to use updated packages.


In [6]:
# split data into training and evaluation
df_train=df.sample(frac=0.95,random_state=200)
df_eval=df.drop(df_train.index)

In [7]:
# save data into .jsonl files
df_train.to_json("ultrachat_chunk_train.jsonl", orient="records", lines=True)
df_eval.to_json("ultrachat_chunk_eval.jsonl", orient="records", lines=True)

In [8]:
%ls /share/pi/nigam/users/aunell/mistral-finetune/data

ultrachat_chunk_eval.jsonl  ultrachat_chunk_train.jsonl


In [12]:
!python -m utils.reformat_data /share/pi/nigam/users/aunell/mistral-finetune/data/ultrachat_chunk_train.jsonl

In [13]:
!python -m utils.reformat_data /share/pi/nigam/users/aunell/mistral-finetune/data/ultrachat_chunk_eval.jsonl

In [22]:
!python -m utils.validate_data --train_yaml example/7B.yaml


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.1.3 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/local-scratch/nigam/users/aunell/miniconda3/envs/mistral/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/local-scratch/nigam/users/aunell/miniconda3/envs/mistral/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/share/pi/nigam/users/aunell/mistral-finetune/utils/validate_data.py", line 17, in <module>
    from finetune.data.dataset import parse_data_sources
  File "/share/pi/nigam/users/aunell/mistral-finetune/fin

# Start Training

In [25]:
# these info is needed for training
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [39]:
# define training configuration
# for your own use cases, you might want to change the data paths, model path, run_dir, and other hyperparameters

config = """
# data
data:
  instruct_data: "/share/pi/nigam/users/aunell/mistral-finetune/data/ultrachat_chunk_train.jsonl"  # Fill
  data: ""  # Optionally fill with pretraining data
  eval_instruct_data: "/share/pi/nigam/users/aunell/mistral-finetune/data/ultrachat_chunk_eval.jsonl"  # Optionally fill

# model
model_id_or_path: "/share/pi/nigam/users/aunell/mistral-finetune/mistral_models"  # Change to downloaded path
lora:
  rank: 64

# optim
# tokens per training steps = batch_size x num_GPUs x seq_len
# we recommend sequence length of 32768
# If you run into memory error, you can try reduce the sequence length
seq_len: 2048
batch_size: 1
num_microbatches: 8
max_steps: 100
optim:
  lr: 1.e-4
  weight_decay: 0.1
  pct_start: 0.05

# other
seed: 0
log_freq: 1
eval_freq: 100
no_eval: False
ckpt_freq: 100

save_adapters: True  # save only trained LoRA adapters. Set to `False` to merge LoRA adapter into the base model and save full fine-tuned model

run_dir: "/share/pi/nigam/users/aunell/mistral-finetune/test_ultra"  # Fill
"""

# save the same file locally into the example.yaml file
import yaml
with open('/share/pi/nigam/users/aunell/mistral-finetune/example/example.yaml', 'w') as file:
    yaml.dump(yaml.safe_load(config), file)


In [36]:
%pip install "numpy<2"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 26.7 MB/s eta 0:00:00:00:01
  Attempting uninstall: numpy
    Found existing installation: numpy 2.1.3
    Uninstalling numpy-2.1.3:
      Successfully uninstalled numpy-2.1.3
Note: you may need to restart the kernel to use updated packages.


In [40]:
!torchrun --nproc-per-node 1 -m train /share/pi/nigam/users/aunell/mistral-finetune/example/7B.yaml

[W socket.cpp:464] [c10d] The server socket cannot be initialized on [::]:29500 (errno: 97 - Address family not supported by protocol).
[W socket.cpp:697] [c10d] The client socket cannot be initialized to connect to [localhost]:29500 (errno: 97 - Address family not supported by protocol).
args: TrainArgs(data=DataArgs(data='', shuffle=False, instruct_data='/share/pi/nigam/users/aunell/mistral-finetune/data/ultrachat_chunk_train.jsonl', eval_instruct_data='/share/pi/nigam/users/aunell/mistral-finetune/data/ultrachat_chunk_eval.jsonl', instruct=InstructArgs(shuffle=True, dynamic_chunk_fn_call=True)), model_id_or_path='/share/pi/nigam/users/aunell/mistral-finetune/mistral_models/7B-v0.3', run_dir='/share/pi/nigam/users/aunell/mistral-finetune/run5', optim=OptimArgs(lr=6e-05, weight_decay=0.1, pct_start=0.05), seed=0, num_microbatches=1, seq_len=32768, batch_size=1, max_norm=1.0, max_steps=300, log_freq=1, ckpt_freq=100, save_adapters=True, no_ckpt=False, num_ckpt_keep=3, eval_freq=100, no

In [42]:
!pip freeze > /share/pi/nigam/users/aunell/mistral-finetune/requirements_alyssa.txt